In [201]:
import pandas as pd

import numpy as np

from datetime import datetime




In [275]:
# df_canceled = pd.read_excel("CDM_cancellation_edited.xlsx")
df_country_code = pd.read_excel('cdm-country-code.xlsx')
df_scope_code = pd.read_excel('cdm-scope-mapping.xlsx')

df_activities= pd.read_excel('CDM-Activities-October.xlsx')
# df_activities['CDM project reference number'] = df_activities['CDM project reference number'].astype(str)
df_activities['CDM project reference number'] = df_activities['CDM project reference number'].fillna(0).astype(int)



more_countries = df_activities[df_activities["List of host countries (ISO 2)"].str.len() > 2]

more_countries.to_excel("CDM_projects_many_countries.xlsx")

more_countries['List of host countries (ISO 2)'].unique()


df_activities["List of host countries (ISO 2)"] = df_activities["List of host countries (ISO 2)"].str[:2]


In [ ]:
df_unep = pd.read_excel("cdm-pipeline.xlsx")

pdd_developer_map = df_unep.set_index('Unique project ID')['PDD Consultant'].to_dict()


In [280]:
df_country_code['geographicIdentifier'] = df_country_code.apply(
    lambda row: "{\"latitude\":" + str(row["latitude"]) + ",\"longitude\":" + str(row["longitude"]) + "}", axis=1
)
print(df_country_code['geographicIdentifier'])

id_to_id = df_activities.set_index('CDM project reference number')['Unique project identifier (traceable with Google)'].to_dict()


# diff = df_country_code['country'].unique()
long_country_codes = df_country_code[df_country_code['country'].str.len() > 2]
print(long_country_codes)

country_name_map = df_country_code.set_index('country')['cadtcountry'].to_dict()
country_name_map['NA']="Namibia"
geography_map = df_country_code.set_index('name')['geographicIdentifier'].to_dict()
verification_body_map = df_activities.set_index('Unique project identifier (traceable with Google)')['Start of first crediting period'].to_dict()



project_status_map = {
    "Registered": "Registered",
    "Rejected": "Withdrawn",
    "Withdrawn":"Withdrawn",
    "WithdrawnBeforePublication": "Withdrawn",
    "Withdrawn Before Publication": "Withdrawn",
    "Deregistered": "De-registered",
    "Pending Publication": "Authorized",
    "Provisional": "Validated",
    "Validation Replaced": "Validated",
    "Requesting Registration": "Listed",
    "Validation Public": "Validated",
    "Validation Terminated": "Withdrawn",
    "NA":"Listed",
    "":"Listed",
    np.nan: "Listed"  # Handling actual NaN values
}

sector_code_map = {
  "1": 'Energy industries (renewable - / non-renewable sources)',
  "2": 'Energy distribution',
  "3": 'Energy demand',
  "4": 'Manufacturing industries',
  "5": 'Chemical industries',
  "6": 'Construction',
  "7": 'Transport',
  "8": 'Mining/mineral production',
  "9": 'Metal production',
  "10": 'Fugitive emissions from fuels (solid, oil and gas)',
  "11": 'Fugitive emissions from production and consumption of halocarbons and sulphur hexafluoride',
  "12": 'Solvent use',
  "13": 'Waste handling and disposal',
  "14": 'Afforestation and reforestation',
  "15": 'Agriculture'
}


def fillna_by_dtype(df):
    for column in df.columns:
        if df[column].dtype == np.number:  # If the column is numeric
            df[column] = df[column].fillna(0)
        elif df[column].dtype == np.object:  # If the column is of object type (e.g., strings)
            df[column] = df[column].fillna('NA')
        elif pd.api.types.is_datetime64_any_dtype(df[column]):  # If the column is datetime
            df[column] = df[column].fillna(pd.Timestamp.min)
    return df
# Apply the function to each column in the DataFrame

df_cadt_country = pd.read_excel("CADTrust_Country.xlsx")

set_cadt_country = set(df_cadt_country['Country'])
set_country_code = set(df_country_code['name'])

# Find non-matching elements
non_matching_in_cadt_country = set_cadt_country - set_country_code
non_matching_in_country_code = set_country_code - set_cadt_country





# print(pdd_developer_map)

# print(id_to_id)
# df_unep

0        {"latitude":42.546245,"longitude":1.601554}
1       {"latitude":23.424076,"longitude":53.847818}
2        {"latitude":33.93911,"longitude":67.709953}
3      {"latitude":17.060816,"longitude":-61.796428}
4      {"latitude":18.220554,"longitude":-63.068615}
                           ...                      
240     {"latitude":15.552727,"longitude":48.516388}
241      {"latitude":-12.8275,"longitude":45.166244}
242    {"latitude":-30.559482,"longitude":22.937506}
243    {"latitude":-13.133897,"longitude":27.849332}
244    {"latitude":-19.015438,"longitude":29.154857}
Name: geographicIdentifier, Length: 245, dtype: object
Empty DataFrame
Columns: [country, latitude, longitude, name, cadtcountry, geographicIdentifier]
Index: []


In [281]:
rename_columns  = {

  "Unique project identifier (traceable with Google)": "originProjectId",
  "Registration project title": "projectName",
  "Type of CDM project: PA/PoA": "program",
  "Sectoral scope number(s)": "sector",
  "Project type (UNEP DTU)": "projectType",
  "Website project status": "projectStatus",
  "Methodologies used at registration": "methodology",
  "DOE": "validationBody",
  "Start of validation": "validationDate",

  "Country subregion":"inCountryRegion",
  "List of host countries (ISO 2)": "country",
  # Estimations
  # "Start of first crediting period":"creditingPeriodStart",
  # "End of first crediting period":"creditingPeriodEnd",
  # "Total CERs issued":"unitCount",

}

df_project = df_activities[rename_columns.keys()].rename(columns=rename_columns)
df_project["registryOfOrigin"]= "CDM UNFCCC"
df_project['unitMetric'] = 'tCO2e'
df_project['methodology'] = "CDM - " + df_project['methodology']
df_project['projectLink'] = 'https://cdm.unfccc.int'
df_project['projectId'] = df_project['originProjectId']
df_project['coveredByNDC'] = "Unknown"
df_project['projectStatusDate'] = datetime.now().strftime('%Y-%m-%d')
df_project['country'] = df_project['country'].map(country_name_map)
df_project['sector'] = df_project['sector'].map(sector_code_map)
df_project['projectDeveloper'] = df_project['projectId'].map(pdd_developer_map)
df_project["geographicIdentifier"]= df_project['country'].map(geography_map)

df_project['description'] = df_project['projectName']
df_project['ndcInformation'] = "NA"


df_project = fillna_by_dtype(df_project)
df_project["projectStatus"]= df_project['projectStatus'].map(project_status_map)

# Output to check the new DataFrame
df_project.to_excel("CDM-Projects-mapped.xlsx")

# df_project['country'].head(10)










/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_25397/2516400818.py:59: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if df[column].dtype == np.number:  # If the column is numeric
/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_25397/2516400818.py:61: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif df[column].dtype == np.object:  # If the column is of object type (e.g., strings)


{'projectId': 'Unique project identifier (traceable with Google)',
 'originProjectId': 'Unique project identifier (traceable with Google)',
 'projectName': 'Registration project title',
 'sector': 'Sectoral scope number(s)',
 'projectType': 'Project type (UNEP DTU)',
 'projectStatus': 'Simplified project status/Website project status',
 'projectStatusDate': 'Date of last information provided on Para. 218 and 219 of the PCP',
 'methodology': 'Methodologies used at registration',
 'validationBody': 'DOE',
 'validationDate': 'Start of validation'}

In [118]:

df_issuances = pd.read_excel("CDM-Issuances-October.xlxs.xlsx")
# df_issuances.drop('DOE', axis=1, inplace=True)
df_issuances.drop('PA/PoA', axis=1, inplace=True)
df_issuances.drop('CP', axis=1, inplace=True)
df_issuances.drop('Monitoring report number', axis=1, inplace=True)
df_issuances.drop('Last_updated', axis=1, inplace=True)
df_issuances.drop('Project type (UNEP DTU)', axis=1, inplace=True)
df_issuances.drop('Project subtype (UNEP DTU)', axis=1, inplace=True)
df_issuances.drop('Units - Total', axis=1, inplace=True)
df_issuances.drop('Issuance date', axis=1, inplace=True)
df_issuances.drop('HostParty', axis=1, inplace=True)


issuances_columns={
    "CDM project reference number":"projectId",
    "DOE":"verificationBody",
    "Issuance process ID":"id",
    "Monitoring report started":"startDate",
    "Monitoring report ended":"endDate"
}
df_issuances['Monitoring report started'] = pd.to_datetime(df_issuances['Monitoring report started'])
df_issuances['Monitoring report ended'] = pd.to_datetime(df_issuances['Monitoring report ended'])
df_issuances['CDM project reference number'] = df_issuances['CDM project reference number'].map(id_to_id)
df_issuances = df_issuances[issuances_columns.keys()].rename(columns=issuances_columns)


df_issuances['verificationReportDate']=df_issuances['projectId'].map(verification_body_map)
df_issuances['verificationApproach']=""

df_issuances = fillna_by_dtype(df_issuances)

df_issuances

/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_25397/643337254.py:31: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if df[column].dtype == np.number:  # If the column is numeric
/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_25397/643337254.py:33: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif df[column].dtype == np.object:  # If the column is of object type (e.g., strings)


,projectId,verificationBody,id,startDate,endDate,verificationReportDate,verificationApproach
0,IBK3RIR7HVE1SZLK7YZMZPZ02UW7TS,KEarth,4KEarth1568615888.93,2017-04-01,2019-07-31,2014-01-01,
1,KZPP9BVTCRTYEUSRYB2L0ZNRJUDF2X,KEarth,4KEarth1568616446.5,2017-04-01,2019-07-31,2014-05-31,
2,P2UF559LX39J2VDPTZ13PAZUO3DRVV,KEarth,4KEarth1585297661.77,2018-01-01,2019-12-31,2011-01-01,
3,AXF2WJ2NOM4CHZGDOJW9S9DINIAQWS,KEarth,4KEarth1596186178.27,2018-01-01,2020-03-21,2013-03-22,
4,JOUI5BA4DLQS9MVXDKRLL1HHFVQ4M3,KEarth,4KEarth1599113457.49,2015-04-01,2017-12-31,2013-10-10,
...,...,...,...,...,...,...,...
12543,3A5UYHPO8SISCX8T4BJ1AMN6SO780J,No info,sergeyf731618828753.52,2016-09-01,2017-09-30,2013-03-15,
12544,3A5UYHPO8SISCX8T4BJ1AMN6SO780J,No info,sergeyf731618828921.37,2017-10-01,2018-12-08,2013-03-15,
12545,D4CRFLKI60AFK14YEHRDNPY16829DF,No info,sergeyf731650363867.4,2016-10-01,2017-06-30,2012-11-01,
12546,BN70H0KER9GB6Y0BPJBJ3Q4IBIXK7C,No info,slenzen1552989570.98,2017-01-01,2019-01-31,2013-12-27,


In [119]:
df_units = pd.read_excel("CDM-Issuances-October.xlxs.xlsx")


units_columns={
    "HostParty":"unitOwner",
    "CDM project reference number":"projectId",
    "Issuance process ID":"issuanceId",
    "DOE":"verificationBody",
    "Issuance process ID":"id",
   "Units - Total":"unitCount",
   "Issuance date":"vintageYear",

}

df_units = df_units[units_columns.keys()].rename(columns=units_columns)
df_units["unitOwner"] = df_units["unitOwner"].map(country_name_map)
df_units["projectId"] = df_units["projectId"].map(id_to_id)
df_units["countryJurisdictionOfOwner"] = df_units["unitOwner"]
df_units["inCountryJurisdictionOfOwner"] = ""
df_units["projectLocationId"] = ""
df_units["geographyIdentifier"] = df_units["unitOwner"].map(geography_map)
df_units["correspondingAdjustmentDeclaration"] = "Unknown"
df_units["unitRegistryLink"] = "http://cdm.unfccc.int/"
df_units["vintageYear"] = df_units["vintageYear"].astype(str).str[:4]
df_units["unitStatus"] = "Held"
df_units["unitType"] = ""
df_units["unitBlockStart"] = ""
df_units["unitBlockEnd"] = ""


df_units = fillna_by_dtype(df_units)

df_units

/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_25397/643337254.py:31: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if df[column].dtype == np.number:  # If the column is numeric
/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_25397/643337254.py:33: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif df[column].dtype == np.object:  # If the column is of object type (e.g., strings)


,unitOwner,projectId,id,verificationBody,unitCount,vintageYear,countryJurisdictionOfOwner,inCountryJurisdictionOfOwner,projectLocationId,geographyIdentifier,correspondingAdjustmentDeclaration,unitRegistryLink,unitStatus,unitType,unitBlockStart,unitBlockEnd
0,India,IBK3RIR7HVE1SZLK7YZMZPZ02UW7TS,4KEarth1568615888.93,KEarth,20519,2020,India,,,"{""latitude"":20.593684,""longitude"":78.96288}",Unknown,http://cdm.unfccc.int/,Held,,,
1,India,KZPP9BVTCRTYEUSRYB2L0ZNRJUDF2X,4KEarth1568616446.5,KEarth,18925,2020,India,,,"{""latitude"":20.593684,""longitude"":78.96288}",Unknown,http://cdm.unfccc.int/,Held,,,
2,India,P2UF559LX39J2VDPTZ13PAZUO3DRVV,4KEarth1585297661.77,KEarth,24595,2021,India,,,"{""latitude"":20.593684,""longitude"":78.96288}",Unknown,http://cdm.unfccc.int/,Held,,,
3,Thailand,AXF2WJ2NOM4CHZGDOJW9S9DINIAQWS,4KEarth1596186178.27,KEarth,43219,2021,Thailand,,,"{""latitude"":15.870032,""longitude"":100.992541}",Unknown,http://cdm.unfccc.int/,Held,,,
4,India,JOUI5BA4DLQS9MVXDKRLL1HHFVQ4M3,4KEarth1599113457.49,KEarth,116637,2022,India,,,"{""latitude"":20.593684,""longitude"":78.96288}",Unknown,http://cdm.unfccc.int/,Held,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12543,India,3A5UYHPO8SISCX8T4BJ1AMN6SO780J,sergeyf731618828753.52,No info,1231148,2022,India,,,"{""latitude"":20.593684,""longitude"":78.96288}",Unknown,http://cdm.unfccc.int/,Held,,,
12544,India,3A5UYHPO8SISCX8T4BJ1AMN6SO780J,sergeyf731618828921.37,No info,1128556,2022,India,,,"{""latitude"":20.593684,""longitude"":78.96288}",Unknown,http://cdm.unfccc.int/,Held,,,
12545,China,D4CRFLKI60AFK14YEHRDNPY16829DF,sergeyf731650363867.4,No info,434350,2022,China,,,"{""latitude"":35.86166,""longitude"":104.195397}",Unknown,http://cdm.unfccc.int/,Held,,,
12546,India,BN70H0KER9GB6Y0BPJBJ3Q4IBIXK7C,slenzen1552989570.98,No info,102514,2020,India,,,"{""latitude"":20.593684,""longitude"":78.96288}",Unknown,http://cdm.unfccc.int/,Held,,,


In [120]:
df_retired = pd.read_excel("CDM_Retired_October.xlsx")
df_retired['Project number'] = df_retired['Project number'].fillna(0).astype(int)
df_retired["Status"] = "Retired"
df_retired["PARTY CODE"] = df_retired['PARTY CODE'].map(country_name_map)


vintage_year_map= df_units.set_index('projectId')['vintageYear'].to_dict()
issuance_id_map= df_units.set_index('projectId')['id'].to_dict()
verif_body_units_map= df_units.set_index('projectId')['verificationBody'].to_dict()
geog_identifier_units_map = df_units.set_index('projectId')['geographyIdentifier'].to_dict()




# df_retired


retired_columns={
    "Status":"unitStatus",
    "Total":"unitTotal",
    "PARTY CODE":"unitOwner",
    "Project number":"projectId",
    "SERIAL RANGE":"SERIAL RANGE"
}



df_retired = df_retired[retired_columns.keys()].rename(columns=retired_columns)
df_retired[['unitBlockStart', 'unitBlockEnd']] = df_retired['SERIAL RANGE'].str.split('-', 1, expand=True)
df_retired['unitBlockStart'] = df_retired['unitBlockStart'].fillna(0).astype(int)
df_retired['unitBlockEnd'] = df_retired['unitBlockEnd'].fillna(0).astype(int)
df_retired.drop('SERIAL RANGE', axis=1, inplace=True)
df_retired['projectId']=df_retired['projectId'].map(id_to_id)
df_retired["countryJurisdictionOfOwner"] = df_retired["unitOwner"]
df_retired["inCountryJurisdictionOfOwner"] = ""
df_retired["unitType"] = ""
df_retired["correspondingAdjustmentDeclaration"] = "Unknown"
df_retired["unitRegistryLink"] = "http://cdm.unfccc.int/"
df_retired["vintageYear"]= df_retired['projectId'].map(vintage_year_map)
df_retired["id"]= df_retired['projectId'].map(issuance_id_map)
df_retired["verificationBody"]= df_retired['projectId'].map(verif_body_units_map)
df_retired['geographyIdentifier']= df_retired['projectId'].map(geog_identifier_units_map)
df_retired["projectLocationId"] = ""
df_retired = fillna_by_dtype(df_retired)





df_retired


/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_25397/643337254.py:31: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if df[column].dtype == np.number:  # If the column is numeric
/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_25397/643337254.py:33: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif df[column].dtype == np.object:  # If the column is of object type (e.g., strings)


,unitStatus,unitTotal,unitOwner,projectId,unitBlockStart,unitBlockEnd,countryJurisdictionOfOwner,inCountryJurisdictionOfOwner,unitType,correspondingAdjustmentDeclaration,unitRegistryLink,vintageYear,id,verificationBody,geographyIdentifier,projectLocationId
0,Retired,2210,Honduras,HG20LFVJPFURJJ4M6E6MS07HTOZTR4,1,2210,Honduras,,,Unknown,http://cdm.unfccc.int/,2015,RWTUV1425040593.92,TUV NORD,"{""latitude"":15.199999,""longitude"":-86.241905}",
1,Retired,7304,Honduras,76B2SQ7Z56786E62H2ISI2HHQNBYX0,2211,9514,Honduras,,,Unknown,http://cdm.unfccc.int/,2014,TUEV-RHEIN1366872726.97,TUV NORD,"{""latitude"":15.199999,""longitude"":-86.241905}",
2,Retired,48230,India,NMHQDVVH5DX47Y8VGVARMLMF76RWAP,1,48230,India,,,Unknown,http://cdm.unfccc.int/,2014,TUEV-SUED1397119754.81,TUV SUD,"{""latitude"":20.593684,""longitude"":78.96288}",
3,Retired,45988,Brazil,T3TA3SUQYRH7QZ2O5HCAO56AD062PE,1,45988,Brazil,,,Unknown,http://cdm.unfccc.int/,2005,SGS-UKL1124438867.71,SGS,"{""latitude"":-14.235004,""longitude"":-51.92528}",
4,Retired,542829,India,YL3CYU9HUXW76LEKIZMWMCVJF1G9E1,48231,591059,India,,,Unknown,http://cdm.unfccc.int/,2013,SGS-UKL1349867953.32,SGS,"{""latitude"":20.593684,""longitude"":78.96288}",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12529,Retired,3241,India,HZ71HCGMVWBSLX4COTU7NONB1ILUBE,325185188,325188428,India,,,Unknown,http://cdm.unfccc.int/,2013,RWTUV1350367048.39,TUV NORD,"{""latitude"":20.593684,""longitude"":78.96288}",
12530,Retired,59442,India,HZ71HCGMVWBSLX4COTU7NONB1ILUBE,325188429,325247870,India,,,Unknown,http://cdm.unfccc.int/,2013,RWTUV1350367048.39,TUV NORD,"{""latitude"":20.593684,""longitude"":78.96288}",
12531,Retired,167997,China,FK6KNM929ANB7LR9JBWHCTVNJOSOHQ,1194113250,1194281246,China,,,Unknown,http://cdm.unfccc.int/,2023,CTI1666763574.07,CTI,"{""latitude"":35.86166,""longitude"":104.195397}",
12532,Retired,540390,Brazil,M2UR4SLALKN8YAZT5ZJ11YR6BDXBLG,211555446,212095835,Brazil,,,Unknown,http://cdm.unfccc.int/,2023,RINA1677171969.86,RINA,"{""latitude"":-14.235004,""longitude"":-51.92528}",


In [135]:
import requests
from requests.exceptions import RequestException
import json

from datetime import datetime


In [321]:

# Define the API endpoint
api_endpoint = "http://localhost:31310/v1/projects"

def datetime_to_string(value):
    if isinstance(value, datetime):
        return value.strftime("%Y-%m-%d")
    return value
# Function to send POST request for each row in the DataFrame
def post_projects(df):
    # Iterate over each row in the DataFrame
    for index, row in df[13074:13173].iterrows():
        # Construct the request body from the row, excluding 'projectLocation'
        # and adding 'projectLocations' which is not in df_project
        request_body = row.to_dict()
        request_body = {k: datetime_to_string(v) for k, v in row.to_dict().items()}
        # Example additional data for 'projectLocations' as it is not present in the DataFrame
        request_body['projectLocations'] = [
            {
                "country": request_body.get('country', 'Unknown'),
                "inCountryRegion": request_body.get('inCountryRegion', 'Unknown'),
                "geographicIdentifier":request_body.get('geographicIdentifier',{})  # replace with actual geographicIdentifier if available
            }
        ]
        
        # Remove the keys that are not needed for the POST request
        request_body.pop('country', None)
        request_body.pop('inCountryRegion', None)
        request_body.pop('geographicIdentifier', None)

       
        # Send POST request
        try:
            response = requests.post(api_endpoint, json=request_body)
            # Check if the request was successful
            if response.status_code == 200:
                print(f"Successfully posted project with ID: {row['originProjectId']}")
            else:
                print(request_body)
                print(f"Failed to post project with ID: {row['originProjectId']}. Status code: {response.status_code}. Message: {response.json()}")
                break
        except RequestException as e:
            print(request_body)
            print(f"Request failed for project with ID: {row['originProjectId']}. Error: {e}")
            break

# Call the function to start posting data
post_projects(df_project)
# df_project

{'originProjectId': 'LGABR9Q0JYH68VN5WI7MUXTEKDCSOP', 'projectName': 'NA', 'program': 'PoA', 'sector': 'NA', 'projectType': 'EE households', 'projectStatus': 'Listed', 'methodology': 'NA', 'validationBody': 'NA', 'validationDate': 'NA', 'registryOfOrigin': 'CDM UNFCCC', 'unitMetric': 'tCO2e', 'projectLink': 'https://cdm.unfccc.int', 'projectId': 'LGABR9Q0JYH68VN5WI7MUXTEKDCSOP', 'coveredByNDC': 'Unknown', 'projectStatusDate': '2023-11-24', 'projectDeveloper': 'NA', 'description': 'NA', 'ndcInformation': 'NA', 'projectLocations': [{'country': 'Namibia', 'inCountryRegion': 'NA', 'geographicIdentifier': '{"latitude":-22.95764,"longitude":18.49041}'}]}
Failed to post project with ID: LGABR9Q0JYH68VN5WI7MUXTEKDCSOP. Status code: 400. Message: {'message': 'Data Validation error', 'errors': ['"validationDate" must be a valid date'], 'success': False}
